#![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)

## Segmentació comportamental basada en clustering (ML no supervisat) 

Aquesta exercici simula l'exercici complet de generació de segments (clusters) de clients utilitzant un dataset d'usuaris i avaluacions de pel·lícules alliberats per una empresa de streaming multimèdia sota demanda per Internet.

*Aquest notebook cobreix:*
* **Part 1:** ETL dels fitxers
* **Part 2:** Enginyeria de característiques
* **Part 3:** Clustering
* **Part 4:** Validació dels resultats
* **Part 5:** Normalització de les dades

## Part 1: ETL de los fitxers

** Resum **

Els arxius que usarem contenen una agregació de 1.000.209 qualificacions anònimes d'aproximadament 3.900 pel·lícules de 6.040 usuaris realitzades durant l'any 2000.

** Fitxer de qualificacions **

Totes les qualificacions estan contingudes en l'arxiu "ratings.dat" i tenen el següent format:

*UserID::MovieID::Rating::Timestamp*

- UserIDs: el rang del qual es troba entre 1 i 6040
- MovieIDs: el rang del qual es troba entre 1 i 3952
- Rating: les qualificacions es realitzen en una escala de 5 estels (sense decimals)
- Timestamp: la marca de temps es representa en segons

**Nota:** Per a cada usuari hi ha com a mínim 20 qualificacions

** Fitxer d'usuaris **

La informació sobre els usuaris està continguda en el fitxer "users.dat" i té el següent format:

*UserID::Gender::Age::Occupation::Zip-code*

Tota la informació demogràfica és proporcionada voluntàriament pels usuaris i no es comprova la seva exactitud. Solament els usuaris que hagin proporcionat dades demogràfiques s'inclouen en aquest conjunt de dades.

- Gender: El gènere es denota per "M" per a homes i "F" per a dones
- Age: Distribuïda en els següents rangs:

	*  1:  "- 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56 - "

- Occupation: Es tria de les següents opcions:

	*  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"
    
** FFitxer de pel·lícules **

La informació es troba en l'arxiu "movies.dat" i està en el següent format:

*MovieID::Title::Genres*

- Title: Els títols són idèntics als títols proporcionats per la IMDB (incloent any de llançament)
- Genres: Els gèneres estan separats per *pipes* i se seleccionen de la següent llista:
    * Action
	* Adventure
	* Animation
	* Children's
	* Comedy
	* Crime
	* Documentary
	* Drama
	* Fantasy
	* Film-Noir
	* Horror
	* Musical
	* Mystery
	* Romance
	* Sci-Fi
	* Thriller
	* War
	* Western
    
- Alguns MovieIDs no corresponen a una pel·lícula a causa d'un duplicat accidental a la seva entrades i/o a entrades de prova
- Les pel·lícules s'introdueixen principalment a mà, per la qual cosa poden existir errors i incoherències

In [0]:
# Aggregate all imports here
import os
import re
from pprint import pprint
from collections import OrderedDict

Començarem per visualitzar una mostra de les dades. Per a això usarem les funcions predefinides en els notebooks de Databricks per explorar el seu sistema d'arxius.

Fer servir `display(dbutils.fs.ls("/databricks-datasets/cs110x/ml-1m/data-001")` per mostrar la descripció de les dades que es van a usar.

In [0]:
BASE_DATA_PATH = "/databricks-datasets/cs110x/ml-1m/data-001"
display(dbutils.fs.ls(BASE_DATA_PATH))

path,name,size
dbfs:/databricks-datasets/cs110x/ml-1m/data-001/README,README,5577
dbfs:/databricks-datasets/cs110x/ml-1m/data-001/movies.dat,movies.dat,171308
dbfs:/databricks-datasets/cs110x/ml-1m/data-001/movies.dat.gz,movies.dat.gz,58049
dbfs:/databricks-datasets/cs110x/ml-1m/data-001/ratings.dat,ratings.dat,24594131
dbfs:/databricks-datasets/cs110x/ml-1m/data-001/ratings.dat.gz,ratings.dat.gz,5816216
dbfs:/databricks-datasets/cs110x/ml-1m/data-001/users.dat,users.dat,134368
dbfs:/databricks-datasets/cs110x/ml-1m/data-001/users.dat.gz,users.dat.gz,40690


Ara usarem la comanda `print dbutils.fs.head` per a visualitzar el contingut dels fitxers "movies.dat", "user.dat" i "ratings.dat"

In [0]:
users_path = os.path.join(BASE_DATA_PATH, "users.dat")
print(dbutils.fs.head(users_path))

[Truncated to first 65536 bytes]
1::F::1::10::48067
2::M::56::16::70072
3::M::25::15::55117
4::M::45::7::02460
5::M::25::20::55455
6::F::50::9::55117
7::M::35::1::06810
8::M::25::12::11413
9::M::25::17::61614
10::F::35::1::95370
11::F::25::1::04093
12::M::25::12::32793
13::M::45::1::93304
14::M::35::0::60126
15::M::25::7::22903
16::F::35::0::20670
17::M::50::1::95350
18::F::18::3::95825
19::M::1::10::48073
20::M::25::14::55113
21::M::18::16::99353
22::M::18::15::53706
23::M::35::0::90049
24::F::25::7::10023
25::M::18::4::01609
26::M::25::7::23112
27::M::25::11::19130
28::F::25::1::14607
29::M::35::7::33407
30::F::35::7::19143
31::M::56::7::06840
32::F::25::0::19355
33::M::45::3::55421
34::F::18::0::02135
35::M::45::1::02482
36::M::25::3::94123
37::F::25::9::66212
38::F::18::4::02215
39::M::18::4::61820
40::M::45::0::10543
41::F::18::4::15116
42::M::25::8::24502
43::M::25::12::60614
44::M::45::17::98052
45::F::45::16::94110
46::M::18::19::75602
47::M::18::4::94305
48::M::25::4::92107
49::M::18::12::77084
50::F::25::2::98133
51::F::1::10::10562
52::M::18::4::72212
53::M::25::0::96931
54::M::50::1::56723
55::F::35::12::55303
56::M::35::20::60440
57::M::18::19::30350
58::M::25::2::30303
59::F::50::1::55413
60::M::50::1::72118
61::M::25::17::95122
62::F::35::3::98105
63::M::18::4::54902
64::M::18::1::53706
65::M::35::12::55803
66::M::25::18::57706
67::F::50::5::60181
68::M::18::4::53706
69::F::25::1::02143
70::M::18::4::53703
71::M::25::14::95008
72::F::45::0::55122
73::M::18::4::53706
74::M::35::14::94530
75::F::1::10::01748
76::M::35::7::55413
77::M::18::4::15321
78::F::45::1::98029
79::F::45::0::98103
80::M::56::1::49327
81::F::25::0::60640
82::M::25::17::48380
83::F::25::2::94609
84::M::18::4::53140
85::M::18::4::94945
86::F::1::10::54467
87::M::25::14::48360
88::F::45::1::02476
89::F::56::9::85749
90::M::56::13::85749
91::M::35::7::07650
92::F::18::4::44243
93::M::25::17::95825
94::M::25::17::28601
95::M::45::0::98201
96::F::25::16::78028
97::F::35::3::66210
98::F::35::7::33547
99::F::1::10::19390
100::M::35::17::95401
101::F::18::3::33314
102::M::35::19::20871
103::M::45::7::92104
104::M::25::12::00926
105::M::45::12::90277
106::F::35::11::79101
107::M::45::18::63129
108::M::25::12::30316
109::M::45::15::92028
110::M::25::2::90803
111::M::35::15::55416
112::M::25::16::97209
113::M::18::12::37032
114::F::25::2::83712
115::M::25::17::28083
116::M::25::17::55744
117::M::25::17::33314
118::M::35::17::22315
119::F::1::10::77515
120::M::25::11::27106
121::M::35::7::75229
122::F::18::4::94305
123::M::35::9::67208
124::M::56::7::91356
125::M::45::14::01701
126::M::18::9::98117
127::F::45::3::01770
128::M::56::6::37922
129::M::25::11::20164
130::M::35::17::50021
131::M::18::4::06520
132::M::25::17::99709
133::F::25::0::55071
134::M::25::0::66212
135::M::18::4::20006
136::M::18::2::21202
137::F::45::6::78758
138::M::18::20::22203
139::F::25::20::45409
140::F::35::1::55107
141::M::35::13::62035
142::M::25::7::10011
143::M::18::3::64043
144::M::25::17::29401
145::M::18::4::19081
146::F::35::20::10954
147::M::18::4::91360
148::M::50::17::57747
149::M::25::1::29205
150::M::35::7::98144
151::F::25::20::85013
152::M::18::4::48104
153::M::1::10::51537
154::M::50::20::94530
155::M::35::12::07470
156::F::45::7::14519
157::M::35::16::12866
158::M::56::7::28754
159::F::45::0::37922
160::M::35::7::13021
161::M::45::16::98107-2117
162::F::18::4::93117
163::M::18::4::85013
164::F::56::13::94566
165::M::18::16::98502
166::M::18::4::92802
167::F::25::11::10022
168::F::50::0::46970
169::M::25::7::55439
170::M::25::11::07002
171::F::50::17::55441
172::M::25::3::07661
173::M::25::0::45237
174::M::25::16::21203
175::F::25::2::95123
176::F::18::3::55016
177::M::50::1::54016
178::M::56::17::53705
179::M::25::0::02135
180::M::45::12::01603
181::M::18::17::33186
182::M::18::4::03052
183::F::45::1::55407
184::F::25::0::19001
185::M::45::0::14468
186::M::18::5::91767
187::F::45::1::94061
188::M::56::16::79930
189::M::18::0::60076
190::M::25::17::55125
191::M::1

In [0]:
ratings_path = os.path.join(BASE_DATA_PATH, "ratings.dat")
print(dbutils.fs.head(ratings_path))

[Truncated to first 65536 bytes]
1::1193::5::978300760
1::661::3::978302109
1::914::3::978301968
1::3408::4::978300275
1::2355::5::978824291
1::1197::3::978302268
1::1287::5::978302039
1::2804::5::978300719
1::594::4::978302268
1::919::4::978301368
1::595::5::978824268
1::938::4::978301752
1::2398::4::978302281
1::2918::4::978302124
1::1035::5::978301753
1::2791::4::978302188
1::2687::3::978824268
1::2018::4::978301777
1::3105::5::978301713
1::2797::4::978302039
1::2321::3::978302205
1::720::3::978300760
1::1270::5::978300055
1::527::5::978824195
1::2340::3::978300103
1::48::5::978824351
1::1097::4::978301953
1::1721::4::978300055
1::1545::4::978824139
1::745::3::978824268
1::2294::4::978824291
1::3186::4::978300019
1::1566::4::978824330
1::588::4::978824268
1::1907::4::978824330
1::783::4::978824291
1::1836::5::978300172
1::1022::5::978300055
1::2762::4::978302091
1::150::5::978301777
1::1::5::978824268
1::1961::5::978301590
1::1962::4::978301753
1::2692::4::978301570
1::260::4::978300760
1::1028::5::978301777
1::1029::5::978302205
1::1207::4::978300719
1::2028::5::978301619
1::531::4::978302149
1::3114::4::978302174
1::608::4::978301398
1::1246::4::978302091
2::1357::5::978298709
2::3068::4::978299000
2::1537::4::978299620
2::647::3::978299351
2::2194::4::978299297
2::648::4::978299913
2::2268::5::978299297
2::2628::3::978300051
2::1103::3::978298905
2::2916::3::978299809
2::3468::5::978298542
2::1210::4::978298151
2::1792::3::978299941
2::1687::3::978300174
2::1213::2::978298458
2::3578::5::978298958
2::2881::3::978300002
2::3030::4::978298434
2::1217::3::978298151
2::3105::4::978298673
2::434::2::978300174
2::2126::3::978300123
2::3107::2::978300002
2::3108::3::978299712
2::3035::4::978298625
2::1253::3::978299120
2::1610::5::978299809
2::292::3::978300123
2::2236::5::978299220
2::3071::4::978299120
2::902::2::978298905
2::368::4::978300002
2::1259::5::978298841
2::3147::5::978298652
2::1544::4::978300174
2::1293::5::978298261
2::1188::4::978299620
2::3255::4::978299321
2::3256::2::978299839
2::3257::3::978300073
2::110::5::978298625
2::2278::3::978299889
2::2490::3::978299966
2::1834::4::978298813
2::3471::5::978298814
2::589::4::978299773
2::1690::3::978300051
2::3654::3::978298814
2::2852::3::978298958
2::1945::5::978298458
2::982::4::978299269
2::1873::4::978298542
2::2858::4::978298434
2::1225::5::978298391
2::2028::4::978299773
2::515::5::978298542
2::442::3::978300025
2::2312::3::978299046
2::265::4::978299026
2::1408::3::978299839
2::1084::3::978298813
2::3699::2::978299173
2::480::5::978299809
2::1442::4::978299297
2::2067::5::978298625
2::1265::3::978299712
2::1370::5::978299889
2::1193::5::978298413
2::1801::3::978300002
2::1372::3::978299941
2::2353::4::978299861
2::3334::4::978298958
2::2427::2::978299913
2::590::5::978299083
2::1196::5::978298730
2::1552::3::978299941
2::736::4::978300100
2::1198::4::978298124
2::593::5::978298517
2::2359::3::978299666
2::95::2::978300143
2::2717::3::978298196
2::2571::4::978299773
2::1917::3::978300174
2::2396::4::978299641
2::3735::3::978298814
2::1953::4::978298775
2::1597::3::978300025
2::3809::3::978299712
2::1954::5::978298841
2::1955::4::978299200
2::235::3::978299351
2::1124::5::978299418
2::1957::5::978298750
2::163::4::978299809
2::21::1::978299839
2::165::3::978300002
2::2321::3::978299666
2::1090::2::978298580
2::380::5::978299809
2::2501::5::978298600
2::349::4::978299839
2::457::4::978299773
2::1096::4::978299386
2::920::5::978298775
2::459::3::978300002
2::1527::4::978299839
2::3418::4::978299809
2::1385::3::978299966
2::3451::4::978298924
2::3095::4::978298517
2::780::3::978299966
2::498::3::978299418
2::2728::3::978298881
2::2002::5::978300100
2::1962::5::978298813
2::1784::5::978298841
2::2943::4::978298372
2::2006::3::978299861
2::318::5::978298413
2::1207::4::978298478
2::1968::2::978298881
2::3678::3::978299250
2::1244::3::978299143
2::356::5::978299686
2::1245::2::978299200
2::1246::5::978299418
2::3893::1::978299535
2::1247::5::978298652
3::3421::4::978

In [0]:
movies_path = os.path.join(BASE_DATA_PATH, "movies.dat")
print(dbutils.fs.head(movies_path))

[Truncated to first 65536 bytes]
1::Toy Story (1995)::Animation|Children's|Comedy
2::Jumanji (1995)::Adventure|Children's|Fantasy
3::Grumpier Old Men (1995)::Comedy|Romance
4::Waiting to Exhale (1995)::Comedy|Drama
5::Father of the Bride Part II (1995)::Comedy
6::Heat (1995)::Action|Crime|Thriller
7::Sabrina (1995)::Comedy|Romance
8::Tom and Huck (1995)::Adventure|Children's
9::Sudden Death (1995)::Action
10::GoldenEye (1995)::Action|Adventure|Thriller
11::American President, The (1995)::Comedy|Drama|Romance
12::Dracula: Dead and Loving It (1995)::Comedy|Horror
13::Balto (1995)::Animation|Children's
14::Nixon (1995)::Drama
15::Cutthroat Island (1995)::Action|Adventure|Romance
16::Casino (1995)::Drama|Thriller
17::Sense and Sensibility (1995)::Drama|Romance
18::Four Rooms (1995)::Thriller
19::Ace Ventura: When Nature Calls (1995)::Comedy
20::Money Train (1995)::Action
21::Get Shorty (1995)::Action|Comedy|Drama
22::Copycat (1995)::Crime|Drama|Thriller
23::Assassins (1995)::Thriller
24::Powder (1995)::Drama|Sci-Fi
25::Leaving Las Vegas (1995)::Drama|Romance
26::Othello (1995)::Drama
27::Now and Then (1995)::Drama
28::Persuasion (1995)::Romance
29::City of Lost Children, The (1995)::Adventure|Sci-Fi
30::Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)::Drama
31::Dangerous Minds (1995)::Drama
32::Twelve Monkeys (1995)::Drama|Sci-Fi
33::Wings of Courage (1995)::Adventure|Romance
34::Babe (1995)::Children's|Comedy|Drama
35::Carrington (1995)::Drama|Romance
36::Dead Man Walking (1995)::Drama
37::Across the Sea of Time (1995)::Documentary
38::It Takes Two (1995)::Comedy
39::Clueless (1995)::Comedy|Romance
40::Cry, the Beloved Country (1995)::Drama
41::Richard III (1995)::Drama|War
42::Dead Presidents (1995)::Action|Crime|Drama
43::Restoration (1995)::Drama
44::Mortal Kombat (1995)::Action|Adventure
45::To Die For (1995)::Comedy|Drama
46::How to Make an American Quilt (1995)::Drama|Romance
47::Seven (Se7en) (1995)::Crime|Thriller
48::Pocahontas (1995)::Animation|Children's|Musical|Romance
49::When Night Is Falling (1995)::Drama|Romance
50::Usual Suspects, The (1995)::Crime|Thriller
51::Guardian Angel (1994)::Action|Drama|Thriller
52::Mighty Aphrodite (1995)::Comedy
53::Lamerica (1994)::Drama
54::Big Green, The (1995)::Children's|Comedy
55::Georgia (1995)::Drama
56::Kids of the Round Table (1995)::Adventure|Children's|Fantasy
57::Home for the Holidays (1995)::Drama
58::Postino, Il (The Postman) (1994)::Drama|Romance
59::Confessional, The (Le Confessionnal) (1995)::Drama|Mystery
60::Indian in the Cupboard, The (1995)::Adventure|Children's|Fantasy
61::Eye for an Eye (1996)::Drama|Thriller
62::Mr. Holland's Opus (1995)::Drama
63::Don't Be a Menace to South Central While Drinking Your Juice in the Hood (1996)::Comedy
64::Two if by Sea (1996)::Comedy|Romance
65::Bio-Dome (1996)::Comedy
66::Lawnmower Man 2: Beyond Cyberspace (1996)::Sci-Fi|Thriller
67::Two Bits (1995)::Drama
68::French Twist (Gazon maudit) (1995)::Comedy|Romance
69::Friday (1995)::Comedy
70::From Dusk Till Dawn (1996)::Action|Comedy|Crime|Horror|Thriller
71::Fair Game (1995)::Action
72::Kicking and Screaming (1995)::Comedy|Drama
73::Mis�rables, Les (1995)::Drama|Musical
74::Bed of Roses (1996)::Drama|Romance
75::Big Bully (1996)::Comedy|Drama
76::Screamers (1995)::Sci-Fi|Thriller
77::Nico Icon (1995)::Documentary
78::Crossing Guard, The (1995)::Drama
79::Juror, The (1996)::Drama|Thriller
80::White Balloon, The (Badkonake Sefid ) (1995)::Drama
81::Things to Do in Denver when You're Dead (1995)::Crime|Drama|Romance
82::Antonia's Line (Antonia) (1995)::Drama
83::Once Upon a Time... When We Were Colored (1995)::Drama
84::Last Summer in the Hamptons (1995)::Comedy|Drama
85::Angels and Insects (1995)::Drama|Romance
86::White Squall (1996)::Adventure|Drama
87::Dunston Checks In (1996)::Children's|Comedy
88::Black Sheep (1996)::Comedy
89::Nick of Time (1995)::Action|Thriller
90::Journey of August King, The (1995)::Drama
92::Mary Reilly (1996)::Drama|Thriller
93::Vampire in Brookly

Carregar els diferents fitxers (usuaris, pel·lícules i qualificacions) en tres RDDs per al seu posterior processament.

In [0]:
N_RDD_ROWS_SAMPLE = 5

In [0]:
rawUsersTextRdd = sc.textFile(users_path)
pprint(rawUsersTextRdd.take(5))

['1::F::1::10::48067',
 '2::M::56::16::70072',
 '3::M::25::15::55117',
 '4::M::45::7::02460',
 '5::M::25::20::55455']

In [0]:
rawRatingsTextRdd = sc.textFile(ratings_path)
pprint(rawRatingsTextRdd.take(N_RDD_ROWS_SAMPLE))

['1::1193::5::978300760',
 '1::661::3::978302109',
 '1::914::3::978301968',
 '1::3408::4::978300275',
 '1::2355::5::978824291']

In [0]:
rawMoviesTextRdd = sc.textFile(movies_path)
pprint(rawMoviesTextRdd.take(N_RDD_ROWS_SAMPLE))

["1::Toy Story (1995)::Animation|Children's|Comedy",
 "2::Jumanji (1995)::Adventure|Children's|Fantasy",
 '3::Grumpier Old Men (1995)::Comedy|Romance',
 '4::Waiting to Exhale (1995)::Comedy|Drama',
 '5::Father of the Bride Part II (1995)::Comedy']

Com t'has adonat observant els fitxers, el fitxer d'usuaris conté algunes línies amb més d'un `zip-code`. Per a simplicitat, eliminarem aquestes línies del fitxer. Per a això crearem una funció que elimini totes les línies que contingui algun caràcter que no sigui '0123456789:MF'.

In [0]:
def isValidLine(line):
    """Verifies if a line is valid to be converted to a dataframe.
    Args:
        line (str): A string.

    Returns:
        boolean: True if valid, False otherwise.
    """    
    full_correct_matching = re.match(
      '^[\d:MF]*$',
      line
    )
    return full_correct_matching is not None

In [0]:
filteredUsersTextRdd = rawUsersTextRdd.filter(lambda x: isValidLine(x))

Com en anteriors ocasions, crea un esquema a mesura per a cadascun dels fitxers.

In [0]:
from pyspark.sql.types import *

# Custom Schema for users
def build_schema(column_type_duples):
  return StructType([ \
    StructField(c[0], c[1], False) for c in column_type_duples
  ])

# UserID::Gender::Age::Occupation::Zip-code
userSchema = build_schema([
  ("UserID", IntegerType()),
  ("Gender", IntegerType()),
  ("Age", IntegerType()),
  ("Occupation", IntegerType()),
  ("Zip-code", IntegerType()),
])
pprint(userSchema)

StructType(List(StructField(UserID,IntegerType,false),StructField(Gender,IntegerType,false),StructField(Age,IntegerType,false),StructField(Occupation,IntegerType,false),StructField(Zip-code,IntegerType,false)))

In [0]:
# UserID::MovieID::Rating::Timestamp
ratingsSchema = build_schema([
  ("UserID", IntegerType()),
  ("MovieID", IntegerType()),
  ("Rating", IntegerType()),
  ("Timestamp", IntegerType()),
])

In [0]:
# MovieID::Title::Genres
moviesSchema = build_schema([
  ("MovieID", IntegerType()),
  ("Title", StringType()),
  ("Genres", StringType()),
])

Elimina les línies no vàlides contingudes en `rawUsersTextRdd`.

In [0]:
filteredUsersTextRdd = rawUsersTextRdd.filter(lambda x: isValidLine(x))

Per poder usar fàcilment diferents algorismes de clustering és convenient que tots els atributs siguin numèrics, per a això convertirem l'atribut gender dels usuaris en 1 si és home ('M') o 0 si és dona ('F'), canvia en l'esquema que has creat anteriorment l'atribut gender a `IntegerType()` si ho tenies com `char` o `string`.

Després transforma el RDD en un DataFrame usant la funció [toDF()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.SQLContext) indicant-li que schema ha d'usar com un paràmetre de la funció (`schema= customSchema`).

Per realitzar aquesta transformació a DataFrame, primer cal convertir cada línia en una llista d'enters, ja que així s'indica en el nostre esquema. Per a això usarem les funcions de Python [split()](https://docs.python.org/2/library/stdtypes.html#str.split) i [int()](https://docs.python.org/2/library/functions.html#int). Pots combinar aquestes dues transformacions juntament amb la conversió de l'atribut `gender` en la mateixa funció lambda o fer una funció amb nom. Després usa una funció `map` per aplicar els canvis a totes les línies del RDD d'usuaris.

Finalment, comprova que les dades mostrades pel comando `display(usersDF)` són correctes.

In [0]:
GENDER_MAP = {
  'M': 1,
  'F': 0,
}
SEPARATOR = '::'

def replace_gender_to_int(line):
  parts = line.split(SEPARATOR)
  parts[1] = GENDER_MAP[parts[1]]
  return [int(p) for p in parts]  


# UserID::Gender::Age::Occupation::Zip-code
integerGendersUsersTextRdd = filteredUsersTextRdd.map(replace_gender_to_int)

usersDF = integerGendersUsersTextRdd.toDF(schema=userSchema)

display(usersDF.head(5))

UserID,Gender,Age,Occupation,Zip-code
1,0,1,10,48067
2,1,56,16,70072
3,1,25,15,55117
4,1,45,7,2460
5,1,25,20,55455


Ara aplica les mateixes transformacions, excepte la de l'atribut gender, al RDD de ratings.

In [0]:
ratingsDF = rawRatingsTextRdd.map(lambda x: [int(c) for c in x.split(SEPARATOR)]).toDF(schema=ratingsSchema)

display(ratingsDF)

UserID,MovieID,Rating,Timestamp
1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
1,1197,3,978302268
1,1287,5,978302039
1,2804,5,978300719
1,594,4,978302268
1,919,4,978301368


## Part 2: Enginyeria de característiques

L'enginyeria de característiques és el procés d'utilitzar el coneixement del domini de les dades per crear les característiques que fan que els algorismes de machine learning treballin de forma correcta. Això és fonamental en l'aplicació del machine learning a dades del món real, i en general és difícil i costosa. L'enginyeria de característiques és un tema informal, però es considera essencial en el machine learning aplicat.

Calcularem unes quantes característiques molt senzilles com el nombre de pel·lícules vistes per cada usuari, la mitjana de les seves qualificacions i la seva variància.

Per obtenir aquestes característiques usarem la funció [groupBy()](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy) dels DataFrames de Spark, així com les funcions d'agregació que ens proporciona SparkQL (https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#moduli-pyspark.sql.functions).

Per simplicitat usarem la funció [àlies()](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SQLContext) per posar els següents noms a les columnes: NumMovies, AvgRating i VarRating.

In [0]:
import pyspark.sql.functions as func

aggRatingsDF = (
  ratingsDF
  .groupBy(ratingsDF.UserID)
  .agg(
    func.count('MovieID').alias('NumMovies'),
    func.avg('Rating').alias('AvgRating'),
    func.variance('Rating').alias('VarRating')
  )
)

aggRatingsDF.show()

+------+---------+------------------+------------------+
UserID|NumMovies| AvgRating| VarRating|
+------+---------+------------------+------------------+
 148| 624| 3.733974358974359|0.9564040828085761|
 463| 123| 3.0|1.5081967213114758|
 471| 105|3.6285714285714286| 1.543406593406593|
 496| 119| 4.294117647058823|0.9042871385842477|
 833| 21|4.0476190476190474|0.9476190476190473|
 1088| 1176|3.3375850340136055| 1.12253509914604|
 1238| 45|3.7111111111111112|0.8919191919191914|
 1342| 92|2.9782608695652173|1.0544672718585761|
 1580| 37|3.5405405405405403| 0.644144144144144|
 1591| 314| 4.340764331210191|0.6598665065830978|
 1645| 522| 4.061302681992337|1.2591906222192808|
 1829| 30|3.3333333333333335|1.1264367816091954|
 1959| 61| 4.508196721311475|0.5207650273224044|
 2122| 208|3.6201923076923075|1.1159188034188035|
 2142| 77|3.5324675324675323|1.0943267259056733|
 2366| 41|3.3902439024390243|0.4439024390243902|
 2659| 161| 4.105590062111801|0.7575310559006208|
 2866| 205| 3.351219512195122|1.3368244858919172|
 243| 33| 3.606060606060606| 1.433712121212121|
 392| 487|3.4743326488706368|1.0523233705985244|
+------+---------+------------------+------------------+
only showing top 20 rows

Ara ajuntarem les característiques que hem extret del fitxer de ratings amb el fitxer d'usuaris. Com no volem haver de preocupar-nos pels valors nuls en les característiques, solament ens quedarem amb els usuaris que hagin qualificat alguna pel·lícula. Per realitzar això, usarem un inner join. Trobarem els detalls d'aquesta funció aquí [join()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html).

In [0]:
usersDF.

In [0]:
joinedDF = usersDF.join(
  aggRatingsDF,
  usersDF.UserID == aggRatingsDF.UserID,
  'inner'
)

display(joinedDF)

UserID,Gender,Age,Occupation,Zip-code,UserID,NumMovies,AvgRating,VarRating
148,1,50,17,57747,148,624,3.733974358974359,0.9564040828085761
463,1,25,7,55105,463,123,3.0,1.5081967213114758
471,1,35,7,8904,471,105,3.6285714285714286,1.543406593406593
496,1,18,4,55455,496,119,4.294117647058823,0.9042871385842477
833,1,35,7,46825,833,21,4.0476190476190474,0.9476190476190473
1088,0,1,10,98103,1088,1176,3.3375850340136055,1.12253509914604
1238,0,35,20,11215,1238,45,3.7111111111111112,0.8919191919191914
1342,1,35,0,94560,1342,92,2.9782608695652173,1.0544672718585761
1580,0,18,4,76201,1580,37,3.5405405405405403,0.644144144144144
1591,1,50,7,26501,1591,314,4.340764331210191,0.6598665065830978


Ara anem a guardar les dades generades per poder reutilitzar-les en un futur sense haver de re-executar tot el notebook, per a això executarem el següent codi.

In [0]:
sqlContext.sql("DROP TABLE IF EXISTS joinedDF")
dbutils.fs.rm("dbfs:/user/hive/warehouse/joinedDF", True)
sqlContext.registerDataFrameAsTable(joinedDF, "joinedDF")

## Part 3: Clustering

Un algorisme d'agrupament (en anglès, clustering) és un procediment d'agrupació d'una sèrie de vectors d'acord amb un criteri. Aquests criteris són en general distancia o similitud. La proximitat es defineix en termes d'una determinada funció de distància, com la d'Euclides, encara que existeixen unes altres mes robustes o que permeten estendre-la a variables discretes.

Existeixen dues grans famílies de clustering:

* *Agrupament jeràrquic*, que pot ser aglomeratiu o divisiu.
* *Agrupament no jeràrquic*, en els quals el nombre de grups es determina per endavant i les observacions es van assignant als grups en funció de la seva proximitat. En aquesta família existeixen una gran quantitat de mètodes, en aquesta PAC usarem el mètode de k-means (k-mitjanes).

### k-means

K-means és un mètode de clustering, que té com a objectiu la partició d'un conjunt de _n_ observacions en _k_ grups en el qual cada observació pertany al grup el valor mitjà del qual és més proper. Un dels avantatges d'aquest mètode és que l'agrupació del conjunt de dades pot il·lustrar-se en una partició de l'espai de dades en [cel·les de Voronoi](https://es.wikipedia.org/wiki/Pol%C3%ADgonos_de_Thiessen#Diagramas_de_Voron.C3.B3i_en_el_plano_euclidiano_.7F.27.22.60UNIQ--postMath-00000001-QINU.60.22.27.7F).


El problema és computacionalment difícil (NP-hard). No obstant això, hi ha heurístiques molt eficients que s'empren comunament i que convergeixen ràpidament a un òptim local.

L'algorisme de k-means més comuna utilitza una tècnica de refinament iteratiu, tal com es descriu a continuació:

Donat un conjunt inicial de _k_ centroides $$ m_1^{(1)},...,m_k^{(1)} $$ l'algorisme contínua alternant entre aquests dos passos:


* *Pas d'asignació:* Assigna cada observació al grup amb la mitjana més propera (és a dir, la partició de les observacions d'acord amb el diagrama de Voronoi generat pels centroides).

$$ S_{i}^{(t)} = \\{ x_p: || x_p - m_i^{(t)} || \leq || x_p - m_j^{(t)} || \forall 1 \leq j \leq k \\} $$

* *Pas d'actualizació:* Calcular els nous centroides com el centroides de les observacions en el grup.
$$ m_i^{(t+1)} = \frac{1}{|S_i^{(t)}|} \sum^{x_j \in S_i^{(t)}} x_j$$

L'algorisme es considera que ha convergit quan les assignacions ja no canvien. Els centroides solen iniciar-se de forma aleatòria.

El següent pas és preparar les dades per aplicar el k-means. Atès que tot el dataset és numèric i consistent, aquesta serà una tasca senzilla i directa.

L'objectiu és utilitzar el mètode de clustering per determinar _k_ usuaris estàndard (mitjana) que representin la totalitat dels usuaris que tenim en el nostre dataset. El primer pas en la construcció del nostre model de clustering és convertir les característiques que hem calculat en el nostre DataFrame a un vector de característiques utilitzant el mètode [pyspark.ml.feature.VectorAssembler()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.feature.VectorAssembler).

El VectorAssembler és una transformació que combina una llista donada de columnes en una únic vector. Aquesta transformació és molt útil quan volem combinar característiques en cru de les dades amb altres generades en aplicar diferents funcions sobre les dades en un únic vector de característiques. Per integrar en un únic vector tota aquesta informació abans d'executar un algorisme d'aprenentatge automàtic, el VectorAssembler pren una llista amb els noms de les columnes d'entrada (llista de strings) i el nom de la columna de sortida (string).

- Convertir la taula SQL `joinedDF` en un `dataframe` anomenat datasetDF usant la funció table del sqlContext
- Establir les columnes d'entrada del VectorAssember: `['Age','NumMovies','AvgRating','VarRating']`
- Establir la columnes de sortida com `"features"`

In [0]:
from pyspark.ml.feature import VectorAssembler

datasetDF = joinedDF

vectorizer = VectorAssembler()
vectorizer.setInputCols(['Age','NumMovies','AvgRating','VarRating'])
vectorizer.setOutputCol('features')

Out[21]: VectorAssembler_50401168b1b0

Guardar en la memòria cau el dataframe `datasetDF` i renombrarlo a clusteringDF

In [0]:
clusteringDF = datasetDF.cache()

- Llegir la documentació i els exemples de [k-means](https://spark.apache.org/docs/1.6.2/ml-clustering.html#k-means)
- Executar la següent cel·la

In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline

kmeans = KMeans()
print(kmeans.explainParams())

distanceMeasure: the distance measure. Supported options: 'euclidean' and 'cosine'. (default: euclidean)
featuresCol: features column name. (default: features)
initMode: The initialization algorithm. This can be either "random" to choose random points as initial cluster centers, or "k-means||" to use a parallel variant of k-means++ (default: k-means||)
initSteps: The number of steps for k-means|| initialization mode. Must be > 0. (default: 2)
k: The number of clusters to create. Must be > 1. (default: 2)
maxIter: max number of iterations (>= 0). (default: 20)
predictionCol: prediction column name. (default: prediction)
seed: random seed. (default: 7969353092125344463)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 0.0001)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)

La següent cel·la està basada en [Spark 3 ML Pipeline API for clustering](https://spark.apache.org/docs/latest/mllib-clustering.html#k-means).

El primer pas és establir els valors dels paràmetres:
- Definir el nombre de clusters (k) com 10
- Definir el nombre màxim d'iteracions a 25
- Definir el random seed a 1

Ara, crearem el [ML Pipeline](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.Pipeline) (flux d'execució) i establirem les fases del pipeline com vectoritzar i posteriorment aplicar l'algorisme de clustering que hem definit.

Finalment, crearem el model entrenant-ho amb el DataFrame `clusteringDF`.

- Llegir la documentació [k-means](https://spark.apache.org/docs/2.0.1/mllib-clustering.html#k-means).
- Completa i executa la següent cel·la, assegura't d'entendre que és el que succeeix.

In [0]:
K_CLUSTERS = 10
MAX_ITERATIONS = 25
SEED = 1

# Now we set the parameters for the method
kmeans = KMeans().setK(K_CLUSTERS).setMaxIter(MAX_ITERATIONS).setSeed(SEED)

clusteringPipeline = Pipeline()

# We will use the new spark.ml pipeline API. If you have worked with scikit-learn this will be very familiar.
clusteringPipeline.setStages([vectorizer, kmeans])

# Let's train on the entire dataset to see what we get
clusteringModel = clusteringPipeline.fit(clusteringDF)

Executar la següent cel·la i observar els centroides que has obtingut. Després respon a les següents preguntes:

- Creïs que els centrodides són significatius?
- Què creïs que ha succeït?

In [0]:
# ['Age','NumMovies','AvgRating','VarRating']
# Shows the result.
centers = clusteringModel.stages[1].clusterCenters()
print("Centroids: ")
for center in centers:
    print(center)

Centroids: 
[30.72275641 83.08814103 3.77634155 1.03119795]
[ 29.88321168 684.00729927 3.39702402 1.10740002]
[ 28.91162791 517.6744186 3.43058414 1.09125773]
[3.06923077e+01 1.35511538e+03 3.29949970e+00 1.16323569e+00]
[ 28.22145329 394.26989619 3.54643029 1.05018317]
[31.54234841 36.23147257 3.7657328 1.10060999]
[ 27.88888889 891.42222222 3.36901051 1.08666947]
[ 30.32435466 141.03367003 3.71940731 1.02152436]
[ 30.81182796 210.9874552 3.66003582 1.05584672]
[ 29.06108597 294.55656109 3.62668844 1.0329288 ]

<!-- 'Age','NumMovies','AvgRating','VarRating' --> 

Tot i que la representació, i per tant la comprensió, d'aquests centroides amb més de 2-3 dimensions és complexa, podem observar alguns patrons a les diferents dimensions. Podem observar com, algunes dimensions, presenten centroides molt propers. Això succeeix amb l'edat, la mitjana de puntuacions i la variància de puntuacions. Concretament, aquestes dimensions oscil·len entre:

- **Edat**: [27.93 - 36.66]. Observem com els centroides tenen una distància màxima de gairebé 10, tot i que les dades del conjunt es mouen entre 1 i 56, un rang molt més ampli.
- **Mitjana de puntuacions**: [3.295 - 3.771] 
- **Variància de puntuacions**: [0.9923 - 1.1018]

A partir d'aquí, podem observar com els centroides es defineixen principalment pel nombre de pel·lícules visualitzades. En el nostre cas, els centroides oscil·len entre 40.53 i 882.26. Aquest fet condicionarà, probablement, els grups (*clusters*) en funció de quantes pel·lícules hagi valorat l'usuari.

Inspecciona visualment si els centroides predits d'alguns dels elements de clusteringDF estan prop dels valors reals i si tenen sentit, després respon les següents preguntes:

- Com afecta el no haver normalitzat les dades? 
- Hi ha algun atribut més important que els altres? En cas que la resposta sigui afirmativa, Com?

Executa la següent cel·la per veure que cluster s'han assignat cadascun dels registres del DataFrame. Els identificadors de cluster comencen per 0.

In [0]:
predictions = clusteringModel.transform(datasetDF)
display( predictions.select(['Age','NumMovies','AvgRating','VarRating','prediction']))

Age,NumMovies,AvgRating,VarRating,prediction
50,624,3.733974358974359,0.9564040828085761,1
25,123,3.0,1.5081967213114758,7
35,105,3.6285714285714286,1.543406593406593,0
18,119,4.294117647058823,0.9042871385842477,7
35,21,4.0476190476190474,0.9476190476190473,5
1,1176,3.3375850340136055,1.12253509914604,3
35,45,3.7111111111111112,0.8919191919191914,5
35,92,2.9782608695652173,1.0544672718585761,0
18,37,3.5405405405405403,0.644144144144144,5
50,314,4.340764331210191,0.6598665065830978,9


Com hem introduït a l'explicació anterior, la diferència de magnituds entre les variables afecta fortament als centroides. Donat que el nombre de pel·lícules valorades contempla un rang molt més ampli que la resta de dimensions, les distàncies entre centroides es veuen molt afectades per aquesta variable. Aquest problema es pot resoldre normalitzant totes les dimensions abans de realitzar la predicció.

Com hem exposat, efectivament, el nombre de pel·lícules (`NumMovies`) és més important que la resta i, com hem dit, afecta els clusters fent que sigui la mètrica dominant mer definir quin grup pertoca a cada usuari.

## Part 5: Normalització de les dades

Una bona practica quan usem mètodes de machine learning basats en distàncies és assegurar-nos que tots els atributs estan en la mateixa escala. Això no succeeix en el nostre dataset ja que cada atribut té un rang diferent.

Prova les següents opcions de normalització i re-executa el notebook amb les dades normalitzades:

* **normalització:** Converteix tots els atributs al rang [0,1] usant la següent fórmula
$$ x' = \frac{x - min}{max-min} $$

* **estandarizació (z-scores):** Converteix tots els atributs en una distribució normal amb mitjana = 0 i variància = 1 usant la següent fórmula
$$ z = \frac{x - \mu}{\sigma}$$

- Tenen ara els centroides més sentit?
- Has aconseguit reduir error? Això implica que ara els centroides estan millor calculats?

In [0]:
# For the following exercise, we'll create two different pipelines: one of every normalisation.
# Through pipelines objects we can get the full transformed datframe. Calling the method 'transform', all transformes will be applied
# to the original dataframe, specially usefull por calculating RSME cost.
vectorizer_raw = VectorAssembler(inputCols=['Age','NumMovies','AvgRating','VarRating'], outputCol='rawFeatures')

In [0]:
from pyspark.ml.feature import MinMaxScaler


normalizer = MinMaxScaler(inputCol='rawFeatures', outputCol='features')

normalizedClusteringPipeline = Pipeline()
normalizedClusteringPipeline.setStages([vectorizer_raw, normalizer, kmeans])

normalizedClusteringModel = normalizedClusteringPipeline.fit(clusteringDF)
normalized_kmeans = normalizedClusteringModel.stages[-1] # K-means is the last stage of our pipeline
normalized_features_df = normalizedClusteringModel.transform(clusteringDF)


In [0]:
from pyspark.ml.feature import StandardScaler


standardizer = StandardScaler(inputCol='rawFeatures', outputCol='features', withMean=True)

standardizePipeline = Pipeline()
standardizePipeline.setStages([vectorizer_raw,  standardizer, kmeans])

standardizedClusteringModel = standardizePipeline.fit(clusteringDF)
standardized_kmeans = standardizedClusteringModel.stages[-1] # K-means is the last stage of our pipeline
standardized_features_df = standardizedClusteringModel.transform(clusteringDF)


In [0]:
kmeans_normalized_models = [
  {
    'desc': 'Dades normalitzades',
    'model': normalized_kmeans
  },
  {
    'desc': 'Dades estandaritzades',
    'model': standardized_kmeans
  },
]

for norm_kmeans in kmeans_normalized_models:
  print(norm_kmeans['desc'])
  centers = norm_kmeans['model'].clusterCenters()
  print("Centroides:")
  for center in centers:
      print(center)
            
  print('\n#####\n')

Dades normalitzades
Centroides:
[0.39709974 0.04415532 0.75529148 0.22258157]
[0.61818182 0.04621807 0.76356147 0.20253132]
[0.3888331 0.05155605 0.63704128 0.34345476]
[0.8929074 0.04015995 0.7849478 0.19327438]
[0.44830467 0.29162798 0.58489725 0.31196845]
[0.37020913 0.04858459 0.55525799 0.5506356 ]
[0.87965091 0.05855763 0.64627849 0.30129358]
[0. 0.0451362 0.66858614 0.34726483]
[0.61818182 0.05286937 0.64521403 0.33985611]
[0.80583614 0.04064279 0.55963687 0.55720779]

#####

Dades estandaritzades
Centroides:
[-0.7445236 -0.15679945 -1.43476851 1.88862334]
[ 1.48443364 -0.29101309 -0.11516809 -0.27060532]
[-0.17895292 1.30696496 -0.30693802 -0.17184366]
[-0.42553783 -0.40772659 0.26376836 0.60791093]
[-0.29107634 -0.33506861 0.85291104 -0.77376767]
[ 1.50319756 -0.35513156 1.15864832 -0.92337529]
[-0.17159039 3.5524403 -0.95062872 0.17818929]
[-0.37928217 -0.28851962 -0.74198223 -0.17686591]
[ 1.15784362 -0.36753242 -0.75408422 1.39164632]
[-2.29943246 -0.29652381 0.13067144 0.05926432]

#####

> Observem com els centroides ara són molt més dispersos en totes les dimensions. Previsiblement, això ens atorgarà el mateix pes a totes les variables i els grups o *clusters* s'assignaran a partir de tota la informació disponible.

Com podem observar a la taula superior, certament hem aconseguit reduir el valor de RMSE. Sabem que, com més baix sigui aquest valor, millor podem considerar el nostre model. Seguint aquesta idea, podem dir que el millor preprocessat ha sigut la normalització de les `features` a tractar.

Tot i això, cal tenir en compte que les distàncies usades per calcular RMSE són absolutes, i no ràtios. Això vol dir que l'escala de les dades originals afecten directament al valor final de la mètrica. Per exemple, en les dades normalitzades [0-1], la distància entre els centroides i els valors sempre serà més baixa que amb les dades en cru del dataset. La utilitat real de normalitzar les dades és evitar que una de les variables que el model tracta tingui més pes (fals) que altres a causa de la magnitud dels valors. Per dir-ho d'una forma planera, el que té sentit és comprar RMSE de diferents prediccions (habitualment de diferents models), però no de diferents prediccions fruit de diferents dades.

En el nostre cas, es centroides sí que estan millor calculats. Aquesta millor no és deu únicament a la reducció de RMSE, sinó que també és a causa de que les dimensions tenen ara el mateix pes. 

Tot plegat, hem pogut observar les diferents necessitats que es resolen mitjançant la normalització. Cal tenir en compte que en la majoria dels casos de processament de dades, com en el nostre cas, no només és desitjable normalitzar, sinó que resulta una necessitat per evitar resultats incorrectes.